# One Table To Rule Them All: Radio

This notebook generates a table of radio components in the CDFS and ELAIS-S1 fields, according to various incarnations of the ATLAS survey. To run it, you will need a MongoDB server with the RGZ database loaded. All other data is fetched from the internet.

In the following cell, specify the MongoDB server details:

In [1]:
MONGO_HOST = 'localhost'
MONGO_PORT = 27017

Next, we will fetch the resources we need.

In [2]:
NORRIS_COMPONENTS_URI = 'http://www.atnf.csiro.au/people/rnorris/papers/n202/tab4.txt'
NORRIS_CROSS_IDENTIFICATIONS_URI = 'http://www.atnf.csiro.au/people/rnorris/papers/n202/tab6.txt'

In [37]:
# Load Norris components.
import requests, io, astropy.io.ascii as asc, astropy.table, pandas
norris_components = astropy.table.Table.from_pandas(
    pandas.read_fwf(
        io.StringIO(
            requests.get(NORRIS_COMPONENTS_URI).text
        ),
        skiprows=[0, 2],
        header=0,
        widths=map(len, [
                '   # ',
                'Name                       ',
                'Radio RA     ',
                'Radio dec  ',
                'err(RA) ',
                'err(dec) ',
                'Peak Flux  ',
                'Int flux   ',
                'Bmaj   ',
                'Bmin   ',
                ' Bpa      ',
                ' rms  ',
            ])
    )
)
norris_components

#,Name,Radio RA,Radio dec,err(RA),err(dec),Peak Flux,Int flux,Bmaj,Bmin,Bpa,rms
str4,str26,str11,str11,float64,float64,float64,float64,float64,float64,float64,float64
C001,ATCDFS_J032602.78-284709.0,3:26:02.785,-28:47:09.0,0.78,0.73,0.7,1.38,8.3,2.6,60.8,79.3
C002,ATCDFS_J032604.15-275659.3,3:26:04.152,-27:56:59.3,0.55,0.9,0.71,1.97,11.5,7.3,-17.0,71.9
C003,ATCDFS_J032605.68-274734.4,3:26:05.685,-27:47:34.4,0.1,0.11,40.81,74.7,5.9,5.6,85.7,119.1
C004,ATCDFS_J032606.95-275332.2,3:26:06.955,-27:53:32.2,0.52,1.19,0.41,0.43,0.0,0.0,-1.0,76.7
C005,ATCDFS_J032611.47-273243.8,3:26:11.475,-27:32:43.8,0.1,0.1,69.65,110.9,5.3,3.3,89.6,156.3
C006,ATCDFS_J032613.70-281717.7,3:26:13.701,-28:17:17.7,0.57,0.79,0.48,0.54,0.0,0.0,0.0,77.7
C007,ATCDFS_J032615.48-284629.2,3:26:15.489,-28:46:29.2,0.34,0.36,0.45,0.71,0.0,0.0,0.0,66.0
C008,ATCDFS_J032615.55-280601.0,3:26:15.557,-28:06:01.0,0.3,0.47,0.73,1.06,0.0,0.0,-1.0,56.9
C009,ATCDFS_J032616.35-280014.6,3:26:16.353,-28:00:14.6,0.18,0.3,1.24,1.66,0.0,0.0,-1.0,60.1


In [196]:
# Load Norris cross-identifications.
# This table has inconsistent tabs, so we will have to convert them to "soft tabs".
def replace_tabs(s, tabstop=8):
    """Convert tabs to spaces."""
    out = ''
    upto = 0
    last = None
    for c in s:
        if c == '\t':
            # Fill up to next tabstop.
            diff = tabstop - upto % tabstop
            if diff == 0:
                diff = tabstop
            out += ' ' * diff
            upto += diff
            last = c
            continue
        
        last = c
        out += c
        upto += 1

    return out

test_input = ('S001	ATCDFS_J032602.78-284709.0	C001	            SWIRE3_J032603.15-284708.5	3:26:02.785	-28:47:09.06	1.4	33.8	21.1	-1.0	-1.0	-1.0						 	4					looks like a group in irac 1')
test_output = ('S001    ATCDFS_J032602.78-284709.0      C001                SWIRE3_J032603.15-284708.5  3:26:02.785     -28:47:09.06    1.4     33.8    21.1    -1.0    -1.0    -1.0                                                    4                                       looks like a group in irac 1')

assert test_output == replace_tabs(test_input)


norris_cross_identifications = astropy.table.Table.from_pandas(
    pandas.read_fwf(
        io.StringIO(
            '\n'.join(map(
                    lambda s: replace_tab(s, 8),
                    requests.get(NORRIS_CROSS_IDENTIFICATIONS_URI).text.split('\r\n'))
             )
        ),
        skiprows=[0, 2],
        header=0,
        widths=[8, 32, 20, 28, 16, 16, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 16, 8, 16]
    )
)
norris_cross_identifications[700:710]

#,Name,Component,SWIRE,Radio RA,Radio dec,20cm,3.6µm,4.5µm,5.8µm,8.0µm,24µm,U,G,R,I,Z,z(sp),ID,Class,Basis
str4,str26,str19,str26,str11,str12,float64,float64,str8,str8,str8,float64,float64,float64,float64,float64,float64,float64,int64,str4,str3
S701,ATCDFS_J033529.42-285156.5,C753,SWIRE3_J033529.46-285154.5,3:35:29.424,-28:51:56.54,1.7,716.0,521.4,194.3,146.7,-1.0,20.6,18.8,17.5,16.7,16.4,--,3,--,--
S702,ATCDFS_J033530.22-281108.4,C754,SWIRE3_J033530.19-281108.5,3:35:30.229,-28:11:08.45,0.8,32.2,38.5,-1.0,56.2,376.3,99.0,99.0,99.0,99.0,99.0,--,1,AGN,b
S703,ATCDFS_J033531.02-272702.2,C755,--,3:35:31.025,-27:27:02.20,26.1,--,--,--,--,--,--,--,--,--,--,--,9,--,--
S704,ATCDFS_J033532.11-274547.6,C756,SWIRE3_J033532.11-274545.2,3:35:32.115,-27:45:47.61,2.2,164.4,84.6,55.3,40.6,-1.0,--,--,--,--,--,--,3,--,--
S705,ATCDFS_J033532.19-284801.0,C757,SWIRE3_J033532.19-284801.1,3:35:32.192,-28:48:01.08,0.9,222.5,417.8,1084.8,2999.4,10830.1,99.0,24.2,22.0,20.7,20.1,--,1,--,--
S706,ATCDFS_J033533.22-280621.8,C758,--,3:35:33.224,-28:06:21.85,0.3,--,--,--,--,--,--,--,--,--,--,--,9,--,--
S707,ATCDFS_J033533.96-273313.4,"C759,C760,C761,C764",SWIRE3_J033533.90-273310.9,3:35:33.963,-27:33:13.46,9.4,451.6,512.5,724.7,966.6,2208.2,--,19.8,19.5,18.9,--,--,3,AGN,b
S708,ATCDFS_J033534.47-284213.3,C763,SWIRE3_J033534.66-284221.7,3:35:34.479,-28:42:13.31,0.5,79.8,53.1,-1.0,44.3,-1.0,99.0,22.7,21.3,20.4,19.9,--,4,--,--
S709,ATCDFS_J033535.20-281729.0,"C762,C765",SWIRE3_J033535.20-281729.0,3:35:35.20,-28:17:29.00,8.4,34.5,25.3,-1.0,-1.0,-1.0,99.0,99.0,99.0,22.6,21.9,--,-1,AGN,a


Now, we can construct the table. We will have the following columns:

- Key
- Component ID (Norris)
- Source ID (Norris)
- Source Name (Norris)
- SWIRE Name (Norris)
- RA (Norris)
- Dec (Norris)
- Component ID (RGZ)
- Zooniverse ID (RGZ)
- SWIRE Name (RGZ-MV)


In [197]:
columns = [
    'Key', 'Component ID (Norris)', 'Source ID (Norris)', 'Source Name (Norris)',
    'SWIRE Name (Norris)', 'RA (Norris)', 'Dec (Norris)'
]

In [212]:
import astropy.coordinates

component_ids_norris = [r['#'] for r in norris_components]

_component_to_source = {}
for r in norris_cross_identifications:
    for component in r['Component'].split(','):
        _component_to_source[component.strip()] = r['#']
source_ids_norris = [_component_to_source[c] for c in component_ids_norris]

_source_to_name = {r['#']:r['Name'] for r in norris_cross_identifications}
source_names_norris = [_source_to_name[s] for s in source_ids_norris]

_source_to_swire_norris = {r['#']:r['SWIRE'] for r in norris_cross_identifications}
swire_names_norris = [_source_to_swire_norris[s] for s in source_ids_norris]

_positions_norris = [astropy.coordinates.SkyCoord(
    ra=r['Radio RA'],
    dec=r['Radio dec'],
    unit=('hourangle', 'deg')) for r in norris_components]
ras_norris = [p.ra.deg for p in _positions_norris]
decs_norris = [p.dec.deg for p in _positions_norris]

keys = list(range(len(component_ids_norris)))

table = astropy.table.Table(data=[keys, component_ids_norris, source_ids_norris, source_names_norris,
                                  swire_names_norris, ras_norris, decs_norris],
                            names=columns)
table

Key,Component ID (Norris),Source ID (Norris),Source Name (Norris),SWIRE Name (Norris),RA (Norris),Dec (Norris)
int32,str4,str4,str26,str32,float64,float64
0,C001,S001,ATCDFS_J032602.78-284709.0,SWIRE3_J032603.15-284708.5,51.5116041667,-28.7858333333
1,C002,S002,ATCDFS_J032604.15-275659.3,--,51.5173,-27.9498055556
2,C003,S003,ATCDFS_J032605.68-274734.4,--,51.5236875,-27.7928888889
3,C004,S004,ATCDFS_J032606.95-275332.2,--,51.5289791667,-27.8922777778
4,C005,S005,ATCDFS_J032611.47-273243.8,--,51.5478125,-27.5455
5,C006,S006,ATCDFS_J032613.70-281717.7,(441298),51.5570875,-28.28825
6,C007,S007,ATCDFS_J032615.48-284629.2,SWIRE3_J032615.41-284630.7,51.5645375,-28.7747777778
7,C008,S008,ATCDFS_J032615.55-280601.0,SWIRE3_J032615.52-280559.8,51.5648208333,-28.1002777778
8,C009,S009,ATCDFS_J032616.35-280014.6,SWIRE3_J032616.31-280014.7,51.5681375,-28.0040555556
